In [1]:
import pandas as pd
import numpy as np
import joblib
import sklearn
# import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb

In [91]:
# data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")
data = pd.read_csv("Data/Training/bilkulfirstaayenge.csv")

In [75]:
randomlist = random.sample(list(data.customer.unique()),int(0.1*len(data.customer.unique())))

In [76]:
testdata = data[data['customer'].isin(randomlist)]

In [77]:
# a = tesdatax[['ind_recommended','activation','customer','merchant']]
# a.to_csv('d_test.csv', index=False)

In [78]:
data = data[~data['customer'].isin(randomlist)]

In [79]:
#select rows whose column value is not in a list
# traindata = data[~data['customer'].isin(randomlist)]
traindata = data

In [80]:
del data

In [9]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(data, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,10622736,60173
1,1537226,9843


In [104]:
tdata = traindata

In [83]:
# tdata = traindata.drop(traindata[(traindata['activation'] == 0) & (traindata['ind_recommended'] == 0)].sample(203153).index)
# tdata = tdata.drop(tdata[(tdata['activation'] == 0) & (tdata['ind_recommended'] == 1)].sample(1333227).index)
# tdata = tdata.drop(tdata[(tdata['activation'] == 1) & (tdata['ind_recommended'] == 0)].sample(35996).index)

In [109]:
def scoring(datax):
    datax['score'] = datax['activation']
    datax.loc[(datax['activation'] == 1) & (datax['ind_recommended'] == 1), 'score'] = 100
    datax.loc[(datax['activation'] == 0) & (datax['ind_recommended'] == 0), 'score'] = 50
    datax.loc[(datax['activation'] == 0) & (datax['ind_recommended'] == 1), 'score'] = -50
    datax.loc[(datax['activation'] == 1) & (datax['ind_recommended'] == 0), 'score'] = -100
    return datax


In [110]:
tdata = scoring(tdata)

In [111]:
X_train, X_test, y_train, y_test = train_test_split(tdata.drop(['customer','merchant','score','activation','ind_recommended'],axis=1), tdata['score'], test_size=0.1, random_state=1)
# rec = sklearn.ensemble.StackingRegressor(estimators=[('lgb', lgb.LGBMRegressor()), ('cbt', cbt.CatBoostRegressor(verbose = False))], final_estimator=xgb.XGBRegressor())
rec = cbt.CatBoostRegressor()

In [112]:
rec.fit(X_train, y_train)
y_pred = rec.predict(X_test)
print("rmse : ",mean_squared_error(y_test, y_pred)**0.5)

Learning rate set to 0.103694
0:	learn: 58.8219977	total: 29.7ms	remaining: 29.6s
1:	learn: 58.3675808	total: 52.4ms	remaining: 26.2s
2:	learn: 57.9748479	total: 77.8ms	remaining: 25.8s
3:	learn: 57.6500332	total: 99.8ms	remaining: 24.8s
4:	learn: 57.3756195	total: 122ms	remaining: 24.3s
5:	learn: 57.1464079	total: 142ms	remaining: 23.6s
6:	learn: 56.9505544	total: 163ms	remaining: 23.1s
7:	learn: 56.7889911	total: 187ms	remaining: 23.2s
8:	learn: 56.6393427	total: 217ms	remaining: 23.9s
9:	learn: 56.5195849	total: 243ms	remaining: 24.1s
10:	learn: 56.4133816	total: 276ms	remaining: 24.9s
11:	learn: 56.3195036	total: 360ms	remaining: 29.6s
12:	learn: 56.2372619	total: 399ms	remaining: 30.3s
13:	learn: 56.1616698	total: 424ms	remaining: 29.9s
14:	learn: 56.0946030	total: 450ms	remaining: 29.6s
15:	learn: 56.0245294	total: 480ms	remaining: 29.5s
16:	learn: 55.9723018	total: 503ms	remaining: 29.1s
17:	learn: 55.9229774	total: 528ms	remaining: 28.8s
18:	learn: 55.8831343	total: 554ms	remai

In [113]:
testdata['predicted_score'] = rec.predict(testdata.drop(['ind_recommended','activation','customer','merchant'],axis=1))

In [114]:
evalu(testdata[['customer','predicted_score','merchant']],testdata[['customer','ind_recommended','activation','merchant']])

Input Files are Correct
   ind_recommended  avg_30d_act
0                0     0.224111
1                1     0.151895
Incremental Activation Rate for Top 10 ranked Merchants(dataset level):  -0.0722155


/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_54842/245156590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['predicted_score'] = df_input['predicted_score'].round(10)


In [52]:
# testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [39]:
name='allfourregressionwcatboostimbalancefine'

In [40]:
import os
os.makedirs('submissions/'+name+'/models', exist_ok=True)

In [41]:
joblib.dump(rec, 'submissions/'+name+'/models/score.joblib')

['submissions/allfourregressionwcatboostimbalancefine/models/score.joblib']

In [42]:
testdata.predicted_score.mean()

1.9987477589695894

In [ ]:
#form1  (rec.round-0.5)*act

In [27]:
def incr_act_top10(input_df,pred_col,cm_key='customer',treated_col='ind_recommended',actual_col='activation'):
    
	#for correcting variable types
    input_df[[treated_col, actual_col, pred_col]] = input_df[[treated_col, actual_col, pred_col]].apply(pd.to_numeric, errors='coerce')
	
    input_df['rank_per_cm1'] = input_df.groupby(cm_key)[pred_col].rank(method='first', ascending=False)
    
    input_df = input_df.loc[input_df.rank_per_cm1 <= 10,:]
    
    agg_df = input_df.groupby(treated_col,as_index=False).agg({actual_col:'mean'})
    agg_df.columns = [treated_col,'avg_30d_act']
    
    print(agg_df)
    recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==1,'avg_30d_act'])
    not_recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==0,'avg_30d_act'])
    
    return (recommended_avg_30d_act-not_recommended_avg_30d_act)

In [28]:
def evalu(df_input, df_round): 
    # round off scores to 10 decimal points
    df_input['predicted_score'] = df_input['predicted_score'].round(10)


    # groupby customer, merchant and max score
    df_input = df_input.groupby(['customer', 'merchant'], as_index = False)['predicted_score'].agg('max')


    # merging predicted file and dependent variable file
    eval_data = pd.merge(df_round,df_input,on=['customer','merchant'],how='inner').drop_duplicates()
    # deleting the rows having null value in predicted_score
    eval_data = eval_data[~(eval_data['predicted_score'].isna())]


    if df_round.shape[0] != eval_data.shape[0]:
        return('Error: Rows are missing in the output file')
    else:
        print('Input Files are Correct')

    final_score = round(incr_act_top10(input_df=eval_data,pred_col='predicted_score',cm_key='customer',treated_col='ind_recommended',actual_col='activation'), 7)

    print('Incremental Activation Rate for Top 10 ranked Merchants(dataset level): ', final_score)